In [1]:
import base64
import flask
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from collections import OrderedDict
from tabulate import tabulate

import sys
sys.path.append('c:\\Peter\\GitHub\\CoB\\')

import general.RMIT_colours as rc

from Course_enhancement_graphs import (
  line_graph_measure_surveys,
  line_graph_program_measure_surveys,
  generate_ces_pd_table,
  line_graph_gtsq_surveys,
  generate_ces_pd_table,
  graphCourseProgramPie
)

from Course_enhancement_functions import (
  get_term_name,
  get_course_pop
)

from general.db_helper_functions import (
  connect_to_postgres_db,
  db_extract_query_to_dataframe
)

In [2]:

set_values = input("Do you want to manually input the semester/term variables [Y/N]: ")
if set_values == 'Y':
  end_year = input("Last year of CES data (Current): ")
  start_year = input("First year of CES data (Recommend -4 years): ")
  
  year = input("Year of Course Enhancement: ")
  semester = input("Semester of Course Enhancement: ")
  
  enrl_year = input("Year of demographics (Current or -1): ")
  enrl_semester = input("Semester of demographics (Current): ")
  
  comments_year = end_year
  comments_semester = input("Semester of comments (Current): ")
else:
  end_year = 2018
  start_year = 2015
  
  year = 2019
  semester = 1
  
  enrl_year = 2018
  enrl_semester = 1
  
  comments_year = end_year
  comments_semester = 1


Do you want to manually input the semester/term variables [Y/N]: N


In [3]:
'''--------------------------------- Connect to Database  ----------------------------'''
# create postgres engine this is the connection to the postgres database
postgres_pw = input("Postgres Password: ")
postgres_user = 'pjryan'
postgres_host = 'localhost'
postgres_dbname = 'postgres'

con_string = "host='{0}' " \
             "dbname='{1}' " \
             "user='{2}' " \
             "password='{3}' " \
             "".format(postgres_host, postgres_dbname, postgres_user, postgres_pw)

postgres_con, postgres_cur = connect_to_postgres_db(con_string)

Postgres Password: London2012


In [4]:
'''------------------------Get Images---------------------'''
# header image
image_filename = 'C:\\Peter\\CoB\\logos\\L&T_Transparent_200.png'  # replace with your own image
logo = base64.b64encode(open(image_filename, 'rb').read())

# ces scale image (3 explanation)
image_filename = 'C:\\Peter\\CoB\\logos\\CES_scale.png'  # replace with your own image
ces_scale_image = base64.b64encode(open(image_filename, 'rb').read())

In [5]:
from Course_enhancement_report_functions import (list_to_text,
                                                 get_gts_questions,
                                                 make_comments_rows,
                                                 get_course_enhancement_list,
                                                 get_course_ces_data,
                                                 get_course_comments,
                                                 get_course_comments_themes,
                                                 get_course_program_ces_data,
                                                 create_school_options,
                                                 create_course_options,
                                                 get_course_data,
                                                 make_program_page,
                                                 make_course_pack,
                                                 make_header_div,
                                                 make_header_div_selector)

In [6]:
'''-------------------------------------------- Create Dataframes -------------------------------------'''
df_ce = get_course_enhancement_list(year, semester,
                                    cur=postgres_cur)
df_schools = df_ce[['school_code', 'school_name']].drop_duplicates()
df_ce_ces = get_course_ces_data(df_ce['course_code_ces'].tolist(),
                                start_year,
                                end_year,
                                cur=postgres_cur)

df_ce_comments = get_course_comments(df_ce['course_code_ces'].tolist(),
                                     comments_year, comments_semester,
                                     cur=postgres_cur)

df_ce_comment_themes = get_course_comments_themes(df_ce['course_code_ces'].tolist(),
                                                  comments_year, comments_semester,
                                                  cur=postgres_cur)

#df_ce_prg_enrl = get_course_prg_enrl(df_ce['course_code'].tolist(),
#                                     enrl_year, enrl_semester,
#                                     cur=postgres_cur, tbl='tbl_course_program_pop', schema='enrolments')

df_ce_prg_ces = get_course_program_ces_data(df_ce['course_code_ces'].tolist(),
                                            start_year,
                                            end_year,
                                            cur=postgres_cur)

In [7]:
# Setup app
app = dash.Dash(__name__)
app.scripts.config.serve_locally = True




# Create app layout
app.layout = html.Div(
  [
    make_header_div_selector(df_ce, df_schools, logo),
    html.Div(
      id='course-pack'
    ),
  ]
)

In [8]:
# Update course options based on school selection
@app.callback(Output('course-dropdown', 'options'),
              [Input('school-dropdown', 'value')])
def update_course_dropdown(school_code):
  return create_course_options(df_ce, school_code)


# Update the data pack based on course selection
@app.callback(
  Output('course-pack', 'children'),
  [Input('course-dropdown', 'value')],
)
def create_page(course_code_ces):
  return make_course_pack(course_code_ces,
                          df_ce,
                          df_ce_ces,
                          df_ce_comments,
                          df_ce_comment_themes,
                          df_ce_prg_ces,
                          logo,
                          ces_scale_image)


# Upload css formats
css_directory = 'H:\\Data\\CoB Database\\pipeline\\static\\'
stylesheets = ['bWLwgP.css', 'remove_undo.css']

@app.server.route('/<stylesheet>')
def serve_stylesheet(stylesheet):
  if stylesheet not in stylesheets:
    raise Exception('"{}" is excluded from the allowed static files'.format(stylesheet))
  return flask.send_from_directory(css_directory, stylesheet)


for stylesheet in stylesheets:
  app.css.append_css({"external_url": "{}".format(stylesheet)})

In [ ]:
if __name__ == '__main__':
  app.run_server(port=8050, host='127.0.0.1', debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET /_dash-component-suites/dash_renderer/react-dom@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET /_dash-component-suites/dash_html_components/bundle.js?v=0.10.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET /_dash-component-suites/dash_core_components/plotly-1.35.2.min.js?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET /_dash-component-suites/dash_core_components/bundle.js?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:08] "GET /_dash-component-suites/dash_renderer/bundle.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:25:09] "GET /_dash-dependencies 

127.0.0.1 - - [06/Dec/2018 14:27:08] "GET /favicon.ico HTTP/1.1" 500 -
[2018-12-06 14:27:08,249] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Program Files (x86)\WinPython\python-3.6.5.amd64\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Program Files (x86)\WinPython\python-3.6.5.amd64\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Program Files (x86)\WinPython\python-3.6.5.amd64\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Program Files (x86)\WinPython\python-3.6.5.amd64\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Program Files (x86)\WinPython\python-3.6.5.amd64\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Program Fi